<a href="https://colab.research.google.com/github/johnifedili/DS-Assignment-Submission-Practice-Repository/blob/master/Copy_of_Copy_of_01_data_cleaning_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas for Exploratory Data Analysis II 

Pandas a very useful Python library for data manipulation and exploration. We have so much more to see!

In this lesson, we'll continue exploring Pandas for EDA. Specifically: 

- Identify and handle missing values with Pandas.
- Implement groupby statements for specific segmented analysis.
- Use apply functions to clean data with Pandas.

We'll implicitly review many functions from our first Pandas lesson along the way!

## Remember the AdventureWorks Cycles Dataset?
<img align="right" src="http://lh6.ggpht.com/_XjcDyZkJqHg/TPaaRcaysbI/AAAAAAAAAFo/b1U3q-qbTjY/AdventureWorks%20Logo%5B5%5D.png?imgmax=800">

Here's the Production.Product table [data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html), which is a description of the fields (columns) in the table (the .csv file we will import below):<br>
- **ProductID** - Primary key for Product records.
- **Name** - Name of the product.
- **ProductNumber** - Unique product identification number.
- **MakeFlag** - 0 = Product is purchased, 1 = Product is manufactured in-house.
- **FinishedGoodsFlag** - 0 = Product is not a salable item. 1 = Product is salable.
- **Color** - Product color.
- **SafetyStockLevel** - Minimum inventory quantity.
- **ReorderPoint** - Inventory level that triggers a purchase order or work order.
- **StandardCost** - Standard cost of the product.
- **ListPrice** - Selling price.
- **Size** - Product size.
- **SizeUnitMeasureCode** - Unit of measure for the Size column.
- **WeightUnitMeasureCode** - Unit of measure for the Weight column.
- **DaysToManufacture** - Number of days required to manufacture the product.
- **ProductLine** - R = Road, M = Mountain, T = Touring, S = Standard
- **Class** - H = High, M = Medium, L = Low
- **Style** - W = Womens, M = Mens, U = Universal
- **ProductSubcategoryID** - Product is a member of this product subcategory. Foreign key to ProductSubCategory.ProductSubCategoryID.
- **ProductModelID** - Product is a member of this product model. Foreign key to ProductModel.ProductModelID.
- **SellStartDate** - Date the product was available for sale.
- **SellEndDate** - Date the product was no longer available for sale.
- **DiscontinuedDate** - Date the product was discontinued.
- **rowguid** - ROWGUIDCOL number uniquely identifying the record. Used to support a merge replication sample.
- **ModifiedDate** - Date and time the record was last updated.


### Import Pandas

In [ ]:
import pandas as pd
import numpy as np # used for linear algebra and random sampling
# used for plotting charts within the notebook (instead of a separate window)
%matplotlib inline

### Read in the dataset

We are using the `read_csv()` method (and the `\t` separator to specify tab-delimited columns).

In [ ]:
# read in the data
# url = '../../01-intro/data/Production.Product.csv'
url = 'https://raw.githubusercontent.com/austinlasseter/pandas_exercises/master/01-intro/data/Production.Product.csv'
prod = pd.read_csv(url, sep='\t')

In [ ]:
# let's check out the first 3 rows 
prod.head(3)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000
2,3,BB Ball Bearing,BE-2349,1,0,NaN,800,600,0.0,0.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000


In [ ]:
# and the number of rows x cols
prod.shape

(504, 25)

In [ ]:
# give a little info about all the columns
prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ProductID              504 non-null    int64  
 1   Name                   504 non-null    object 
 2   ProductNumber          504 non-null    object 
 3   MakeFlag               504 non-null    int64  
 4   FinishedGoodsFlag      504 non-null    int64  
 5   Color                  256 non-null    object 
 6   SafetyStockLevel       504 non-null    int64  
 7   ReorderPoint           504 non-null    int64  
 8   StandardCost           504 non-null    float64
 9   ListPrice              504 non-null    float64
 10  Size                   211 non-null    object 
 11  SizeUnitMeasureCode    176 non-null    object 
 12  WeightUnitMeasureCode  205 non-null    object 
 13  Weight                 205 non-null    float64
 14  DaysToManufacture      504 non-null    int64  
 15  Produc

### Reset our index (like last time)

Let's bring our `ProductID` column into the index since it's the PK (primary key) of our table and that's where PKs belong as a best practice.

In [ ]:
# replace auto-generated index with the ProductID column
prod.set_index('ProductID', inplace=True)

## Handling missing data

Recall missing data is a systemic, challenging problem for data scientists. Imagine conducting a poll, but some of the data gets lost, or you run out of budget and can't complete it! 😮<br><br>

"Handling missing data" itself is a broad topic. We'll focus on two components:

- Using Pandas to identify we have missing data
- Strategies to fill in missing data (known in the business as `imputing`)
- Filling in missing data with Pandas


### Identifying missing data

Before *handling*, we must identify we're missing data at all!

We have a few ways to explore missing data, and they are reminiscient of our Boolean filters.

In [ ]:
# True when data isn't missing
prod.notnull().head(3)

,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,,,,
1,True,True,True,True,False,True,True,True,True,False,False,False,False,True,False,False,False,False,False,True,False,False,True,True
2,True,True,True,True,False,True,True,True,True,False,False,False,False,True,False,False,False,False,False,True,False,False,True,True
3,True,True,True,True,False,True,True,True,True,False,False,False,False,True,False,False,False,False,False,True,False,False,True,True


In [ ]:
# True when data is missing
prod.isnull().head(3)

,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,,,,
1,False,False,False,False,True,False,False,False,False,True,True,True,True,False,True,True,True,True,True,False,True,True,False,False
2,False,False,False,False,True,False,False,False,False,True,True,True,True,False,True,True,True,True,True,False,True,True,False,False
3,False,False,False,False,True,False,False,False,False,True,True,True,True,False,True,True,True,True,True,False,True,True,False,False


Now, we may want to see null values in aggregate. We can use `sum()` to sum down a given column

In [ ]:
# here is a quick and dirty way to do it
prod.isnull().sum()

Name                       0
ProductNumber              0
MakeFlag                   0
FinishedGoodsFlag          0
Color                    248
SafetyStockLevel           0
ReorderPoint               0
StandardCost               0
ListPrice                  0
Size                     293
SizeUnitMeasureCode      328
WeightUnitMeasureCode    299
Weight                   299
DaysToManufacture          0
ProductLine              226
Class                    257
Style                    293
ProductSubcategoryID     209
ProductModelID           209
SellStartDate              0
SellEndDate              406
DiscontinuedDate         504
rowguid                    0
ModifiedDate               0
dtype: int64

Look! We've found missing values!

How could this missing data be problematic for our analysis?

### Understanding missing data

Finding missing data is the easy part! Determining way to do next is more complicated.

Typically, we are most interested in knowing **why** we are missing data. Once we know what 'type of missingness' we have (the source of missing data), we can proceed effectively.

Let's first quantify how much data we are missing. Here is another implementation of `prod.isnull().sum()`, only wrapped with a `DataFrame` and some labels to make it a little more user-friendly:

In [ ]:
# or we can make things pretty as follows
null_df = pd.DataFrame(prod.isnull().sum(), columns=['Count of Nulls'])
null_df.index.name = 'Column'
null_df.sort_values(['Count of Nulls'], ascending=False).head(10)

,Count of Nulls
Column,
DiscontinuedDate,504
SellEndDate,406
SizeUnitMeasureCode,328
Weight,299
WeightUnitMeasureCode,299
Size,293
Style,293
Class,257
Color,248


### Filling in missing data

How we fill in data depends largely on why it is missing (types of missingness) and what sampling we have available to us.

We may:

- Delete missing data altogether
- Fill in missing data with:
    - The average of the column
    - The median of the column
    - A predicted amount based on other factors
- Collect more data:
    - Resample the population
    - Followup with the authority providing data that is missing


In [ ]:
# describe a particular column with frequencies of categories
prod['Color'].value_counts().sort_index()

Black           93
Blue            26
Grey             1
Multi            8
Red             38
Silver          43
Silver/Black     7
White            4
Yellow          36
Name: Color, dtype: int64

In our case, let's focus on handling missing values in `Color`. Let's get a count of the unique values in that column. We will need to use the `dropna=False` kwarg, otherwise the `pd.Series.value_counts()` method will not count `NaN` (null) values.

In [ ]:
# notice the use of `dropna`
prod['Color'].value_counts(dropna=False)

NaN             248
Black            93
Silver           43
Red              38
Yellow           36
Blue             26
Multi             8
Silver/Black      7
White             4
Grey              1
Name: Color, dtype: int64

Ahoy! We have 248 nulls!

Option 1: Drop the missing values.

In [ ]:
# drops rows where any row has a missing value - this does not happen *in place*, so we are not actually dropping
prod['Color'].dropna(inplace=False)

ProductID
317     Black
318     Black
319     Black
320    Silver
321    Silver
        ...  
992     Black
993     Black
997     Black
998     Black
999     Black
Name: Color, Length: 256, dtype: object

**Important!** `pd.DataFrame.dropna()` and `pd.Series.dropna()` are very versatile! Let's look at the docs (Series is similar):

```python
Signature: pd.DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)
Docstring:
Remove missing values.

See the :ref:`User Guide <missing_data>` for more on which values are
considered missing, and how to work with missing data.

Parameters
----------
axis : {0 or 'index', 1 or 'columns'}, default 0
    Determine if rows or columns which contain missing values are
    removed.

    * 0, or 'index' : Drop rows which contain missing values.
    * 1, or 'columns' : Drop columns which contain missing value.

    .. deprecated:: 0.23.0: Pass tuple or list to drop on multiple
    axes.
how : {'any', 'all'}, default 'any'
    Determine if row or column is removed from DataFrame, when we have
    at least one NA or all NA.

    * 'any' : If any NA values are present, drop that row or column.
    * 'all' : If all values are NA, drop that row or column.
thresh : int, optional
    Require that many non-NA values.
subset : array-like, optional
    Labels along other axis to consider, e.g. if you are dropping rows
    these would be a list of columns to include.
inplace : bool, default False
    If True, do operation inplace and return None.
```

**how**: This tells us if we want to remove a row if _any_ of the columns have a null, or _all_ of the columns have a null.<br>
**subset**: We can input an array here, like `['Color', 'Size', 'Weight']`, and it will only consider nulls in those columns. This is very useful!<br>
**inplace**: This is if you want to mutate (change) the source dataframe. Default is `False`, so it will return a _copy_ of the source dataframe.

To accomplish the same thing, but implement it on our entire dataframe, we can do the following:

In [ ]:
# drops all nulls from the Color column, but returns the entire dataframe instead of just the Color column
prod.dropna(subset=['Color']).head(3)

Column,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,,,,
317,LL Crankarm,CA-5965,0,0,Black,500,375,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,L,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{3C9D10B7-A6B2-4774-9963-C19DCEE72FEA},2014-02-08 10:01:36.827000000
318,ML Crankarm,CA-6738,0,0,Black,500,375,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,M,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{EABB9A92-FA07-4EAB-8955-F0517B4A4CA7},2014-02-08 10:01:36.827000000
319,HL Crankarm,CA-7457,0,0,Black,500,375,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{7D3FD384-4F29-484B-86FA-4206E276FE58},2014-02-08 10:01:36.827000000


Option 2: Fill in missing values

Traditionally, we fill missing data with a median, average, or mode (most frequently occurring). For `Color`, let's replace the nulls with the string value `NoColor`.

Let's first look at the way we'd do it with a single column, using the `pd.Series.fillna()` method:

In [ ]:
prod['Color'].fillna(value='NoColor').head(10)

ProductID
1      NoColor
2      NoColor
3      NoColor
4      NoColor
316    NoColor
317      Black
318      Black
319      Black
320     Silver
321     Silver
Name: Color, dtype: object

Now let's see how we'd do it to the whole dataframe, using the `pd.DataFrame.fillna()` method. Notice the similar API between the methods with the `value` kwarg. Good congruent design, pandas development team! The full dataframe is returned, and not just a column.

In [ ]:
prod.fillna(value={'Color': 'NoColor'}).head(10)

Column,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,,,,
1,Adjustable Race,AR-5381,0,0,NoColor,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
2,Bearing Ball,BA-8327,0,0,NoColor,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000
3,BB Ball Bearing,BE-2349,1,0,NoColor,800,600,0.0,0.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000
4,Headset Ball Bearings,BE-2908,0,0,NoColor,800,600,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B},2014-02-08 10:01:36.827000000
316,Blade,BL-2036,1,0,NoColor,800,600,0.0,0.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{E73E9750-603B-4131-89F5-3DD15ED5FF80},2014-02-08 10:01:36.827000000
317,LL Crankarm,CA-5965,0,0,Black,500,375,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,L,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{3C9D10B7-A6B2-4774-9963-C19DCEE72FEA},2014-02-08 10:01:36.827000000
318,ML Crankarm,CA-6738,0,0,Black,500,375,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,M,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{EABB9A92-FA07-4EAB-8955-F0517B4A4CA7},2014-02-08 10:01:36.827000000
319,HL Crankarm,CA-7457,0,0,Black,500,375,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{7D3FD384-4F29-484B-86FA-4206E276FE58},2014-02-08 10:01:36.827000000
320,Chainring Bolts,CB-2903,0,0,Silver,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{7BE38E48-B7D6-4486-888E-F53C26735101},2014-02-08 10:01:36.827000000


But wait! There's more! We can reference any other data or formulas we want with the imputation (the value we fill the nulls with). This is very handy if you want to impute with the average or median of that column... or even another column altogether! Here are two examples where we will the nulls of `Weight` with the average value from the same column and then another column. This has no practical value in this application, but immense value in other applications.

In [ ]:
# use an average value from the same column 
ave = prod['Weight'].mean()
print(ave)


74.06921951219512


In [ ]:
# use an average value from the same column 
prod['Weight2']=prod['Weight'].copy()
prod.fillna(value={'Weight2': ave}, inplace=True)
prod[['Weight', 'Weight2']].head(3)

Column,Weight,Weight2
ProductID,,
1,NaN,74.06922
2,NaN,74.06922
3,NaN,74.06922


In [ ]:
# use an average value from another column 
ave = prod['ListPrice'].mean()
prod['Weight3']=prod['Weight'].copy()
prod.fillna(value={'Weight3': ave}, inplace=True)
prod[['Weight', 'Weight3']].head(3)

Column,Weight,Weight3
ProductID,,
1,NaN,438.66625
2,NaN,438.66625
3,NaN,438.66625


### More tricks about missing data

In [ ]:
# A method for looking at an individual
prod['Style'].sample(3)

ProductID
744     U 
875     U 
413    NaN
Name: Style, dtype: object

In [ ]:
# I can use indexing to call an individual cell
prod['Style'][744]

'Nationals'

In [ ]:
# But ... what if I want to manipulate that cell?
prod['Style'][744] = 'Nationals'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# There's another way to call a single cell
prod.loc[424, 'Style'] = 'Unisex'
prod.loc[424, 'Style'] 

'Unisex'

In [ ]:
# but... what if it's a null value?
prod.loc[2, 'Style'] 

nan

In [ ]:
# what is the cell type?
type(prod.loc[2, 'Style'] )

float

In [ ]:
# a few methods for testing for missing data
pd.isnull(prod.loc[2, 'Style'] )

True

In [ ]:
pd.isna(prod.loc[2, 'Style'] )

True

In [ ]:
np.isnan(prod.loc[2, 'Style'] )

True

In [ ]:
# Let's reassign as "None"
prod.loc[2, 'Style'] = None
prod.loc[2, 'Style'] 

In [ ]:
type(prod.loc[2, 'Style'] )

NoneType

In [ ]:
pd.isnull(prod.loc[2, 'Style'] )

True

In [ ]:
prod.loc[2, 'Style'] == np.nan
prod.loc[2, 'Style'] 

In [ ]:
type(prod.loc[2, 'Style'] )

NoneType

Important points:

- Don't forget to use the `inplace=True` kwarg to mutate the source dataframe (i.e. 'save changes'). 
- It is helpful to not use `inplace=True` initially to ensure your code/logic is correct, prior to making permanent changes.

## Groupby Statements

In Pandas, groupby statements are similar to pivot tables in that they allow us to segment our population to a specific subset.

For example, if we want to know the average number of bottles sold and pack sizes per city, a groupby statement would make this task much more straightforward.


To think how a groupby statement works, think about it like this:

- **Split:** Separate our DataFrame by a specific attribute, for example, group by `Color`
- **Combine:** Put our DataFrame back together and return some _aggregated_ metric, such as the `sum`, `count`, or `max`.

![](http://i.imgur.com/yjNkiwL.png)

Let's try it out!

Let's group by `Color`, and get a count of products for each color.

In [ ]:
# group by Color, giving the average of each color
prod.groupby(['Color']).mean()

Column,MakeFlag,FinishedGoodsFlag,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Weight,DaysToManufacture,ProductSubcategoryID,ProductModelID,DiscontinuedDate,Weight2,Weight3
Color,,,,,,,,,,,,,
Black,0.774194,0.956989,285.677419,214.258065,415.446239,725.121075,123.021250,1.774194,11.292135,29.247191,NaN,107.756638,221.448185
Blue,0.846154,1.000000,223.692308,167.769231,567.159477,923.679231,17.610909,2.346154,11.115385,21.923077,NaN,26.296803,82.388654
Grey,0.000000,1.000000,4.000000,3.000000,51.562500,125.000000,NaN,0.000000,35.000000,120.000000,NaN,74.069220,438.666250
Multi,0.000000,1.000000,4.000000,3.000000,34.031775,59.865000,NaN,0.000000,19.625000,10.250000,NaN,74.069220,438.666250
Red,0.973684,1.000000,276.421053,207.315789,858.178055,1401.950000,10.173784,2.552632,8.131579,19.789474,NaN,11.855243,21.449901
Silver,0.744186,0.837209,388.465116,291.348837,466.512751,850.305349,40.579412,2.000000,7.027778,31.361111,NaN,47.588906,123.899913
Silver/Black,0.000000,1.000000,500.000000,375.000000,28.424286,64.018571,187.333333,1.000000,13.000000,64.142857,NaN,171.152746,223.238036
White,0.000000,1.000000,4.000000,3.000000,3.379300,9.245000,NaN,0.000000,23.000000,21.000000,NaN,74.069220,438.666250
Yellow,0.888889,1.000000,244.888889,183.666667,597.434781,959.091389,13.820625,2.416667,9.527778,23.888889,NaN,20.514913,61.025694


What do we notice about this output? Are all columns the same? Why or why not?

We can see that the `.count()` method excludes nulls, and there is no way to change this with the current implementation:
```python
Signature: .count()
Docstring: Compute count of group, excluding missing values 
File:      ~/miniconda3/envs/ga/lib/python3.7/site-packages/pandas/core/groupby/groupby.py
Type:      method
```

As a best practice, you should either:
- fill in nulls prior to your .count(), or
- use the PK (primary key) of the table, which is guaranteed non-null

In [ ]:
# group by Color, giving the average price of each color
prod.groupby(['Color'])['ListPrice'].mean()

Color
Black            725.121075
Blue             923.679231
Grey             125.000000
Multi             59.865000
Red             1401.950000
Silver           850.305349
Silver/Black      64.018571
White              9.245000
Yellow           959.091389
Name: ListPrice, dtype: float64

Let's find out the most expensive price for an item, by `Color`:

In [ ]:
prod[['Color', 'ListPrice']].groupby('Color').max().sort_values('ListPrice', ascending=False)

Column,ListPrice
Color,
Red,3578.27
Silver,3399.99
Black,3374.99
Blue,2384.07
Yellow,2384.07
Grey,125.00
Multi,89.99
Silver/Black,80.99
White,9.50


We can also do multi-level groupbys. This is referred to as a `Multiindex` dataframe. Here, we can see the following fields in a nested group by, with a count of Name (with nulls filled!); effectively giving us a count of the number of products for every unique Class/Style combination:

- Class - H = High, M = Medium, L = Low
- Style - W = Womens, M = Mens, U = Universal

In [ ]:
prod.fillna(value={'Name': 'x'}).groupby(by=['Class', 'Style']).count()[['Name']]

Column       Name
Class Style      
H     U        64
L     U        68
M     U        22
      W        22

We can also use the `.agg()` method with multiple arguments, to simulate a `.describe()` method like we used before:

In [ ]:
prod.groupby('Color')['ListPrice'].agg(['count', 'mean', 'min', 'max'])

,count,mean,min,max
Color,,,,
Black,93,725.121075,0.00,3374.99
Blue,26,923.679231,34.99,2384.07
Grey,1,125.000000,125.00,125.00
Multi,8,59.865000,8.99,89.99
Red,38,1401.950000,34.99,3578.27
Silver,43,850.305349,0.00,3399.99
Silver/Black,7,64.018571,40.49,80.99
White,4,9.245000,8.99,9.50
Yellow,36,959.091389,53.99,2384.07


## Apply functions for column operations

Apply functions allow us to perform a complex operation across an entire columns highly efficiently.

For example, let's say we want to change our colors from a word, to just a single letter. How would we do that?

The first step is writing a function, with the argument being the value we would receive from each cell in the column. This function will mutate the input, and return the result. This result will then be _applied_ to the source dataframe (if desired).

In [ ]:
# what colors are there?
prod['Color'].unique()

array([nan, 'Black', 'Silver', 'Red', 'White', 'Blue', 'Multi', 'Yellow',
       'Grey', 'Silver/Black'], dtype=object)

In [ ]:
# write a function to map a single color to a new value
def color_converter(row):
    if (row=='Red') | (row=='White') :
        return 'Stripes'
    elif row=='Blue':
        return 'Stars'
    else:
        return row

In [ ]:
# apply that function
prod['NewColor'] = prod['Color'].apply(color_converter)
prod[['Color','NewColor']].sample(10)

Column,Color,NewColor
ProductID,,
761,Red,Stripes
460,NaN,NaN
756,Red,Stripes
349,NaN,NaN
934,NaN,NaN
721,Red,Stripes
459,NaN,NaN
937,Silver/Black,Silver/Black
938,Silver/Black,Silver/Black


In [ ]:
def color_to_letter(color):
    # this maps the original color to the color we want it to be
    color_dict = {
        'Black': 'B',
        'Silver': 'S',
        'Red': 'R',
        'White': 'W',
        'Blue': 'B',
        'Multi': 'M',
        'Yellow': 'Y',
        'Grey': 'G',
        'Silver/Black': 'V'
    }
    try:
        return color_dict[color]
    # this catches nulls, or any other color we haven't
    # defined in our color_dict, and fills it with 'N'
    except KeyError:
        return 'N'

Now we can _apply_ this function to our `pd.Series` object, returning the result (which we can use to overwrite the source, if we choose).

In [ ]:
prod['NewColor'] = prod['Color'].apply(color_to_letter)
prod[['Color','NewColor']].sample(10)

Column,Color,NewColor
ProductID,,
992,Black,B
369,NaN,N
989,Black,B
924,Black,B
714,Multi,M
817,Black,B
776,Black,B
439,NaN,N
438,NaN,N


The `pd.DataFrame.apply` implementation is similar, however it effectively 'scrolls through' the columns and passes each one sequentially to your function:

```python
Objects passed to the function are Series objects whose index is
either the DataFrame's index (``axis=0``) or the DataFrame's columns
(``axis=1``).
```

It should only be used when you wish to apply the same function to all columns (or rows) of your `pd.DataFrame` object.

We can also use `pd.Series.apply()` with a **labmda expression**. This is an undeclared function and is commonly used for simple functions within the `.apply()` method. Let's use it to add $100 to our `ListPrice` column. Hey, baby needs new shoes!

In [ ]:
# without apply
prod['ListPrice'].tail(10)

ProductID
990    539.99
991    539.99
992    539.99
993    539.99
994     53.99
995    101.24
996    121.49
997    539.99
998    539.99
999    539.99
Name: ListPrice, dtype: float64

In [ ]:
# and now with 100 more dollars!
prod['ListPrice2'] = prod['ListPrice'].apply(lambda x: x + 100)
prod[['ListPrice','ListPrice2']].tail(10)

Column,ListPrice,ListPrice2
ProductID,,
990,539.99,639.99
991,539.99,639.99
992,539.99,639.99
993,539.99,639.99
994,53.99,153.99
995,101.24,201.24
996,121.49,221.49
997,539.99,639.99
998,539.99,639.99


## Wrap up

We've covered even more useful information! Here are the key takeaways:

- **Missing data** comes in many shapes and sizes. Before deciding how to handle it, we identify it exists. We then derive how the missingness is affecting our dataset, and make a determination about how to fill in values.

```python
# pro tip for identifying missing data
df.isnull().sum()
```

- **Grouby** statements are particularly useful for a subsection-of-interest analysis. Specifically, zooming in on one condition, and determining relevant statstics.

```python
# group by 
df.groupby('column').agg['count', 'mean', 'max', 'min']
```

- **Apply functions** help us clean values across an entire DataFrame column. They are *like* a for loop for cleaning, but many times more efficient. They follow a common pattern:
1. Write a function that works on a single value
2. Test that function on a single value
3. Apply that function to a whole column

(The most confusing part of apply functions is that we write them with *a single value* in mind, and then apply them to many single values at once.)